# SpaceX Launch Analysis Dashboard

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)



![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)

In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

In [2]:
# Load the SpaceX launch dataset
spacex_df = pd.read_csv('spacex_launch_dash.csv')

In [3]:
# Create a Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={0: '0', 10000: '10000'},
        value=[0, 10000]
    ),
    
    dcc.Graph(id='success-pie-chart'),
    
    dcc.Graph(id='success-payload-scatter-chart')
])


In [4]:
# Define the callback function for the pie chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')]
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        all_sites_success_counts = spacex_df['class'].sum()
        all_sites_failure_counts = len(spacex_df) - all_sites_success_counts
        fig = go.Figure(data=[go.Pie(labels=['Success', 'Failure'], values=[all_sites_success_counts, all_sites_failure_counts])])
        fig.update_layout(title='Total Success and Failure Counts for All Sites')
        return fig
    else:
        site_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_success_counts = site_df['class'].sum()
        site_failure_counts = len(site_df) - site_success_counts
        fig = go.Figure(data=[go.Pie(labels=['Success', 'Failure'], values=[site_success_counts, site_failure_counts])])
        fig.update_layout(title=f'Success and Failure Counts for {entered_site}')
        return fig



In [5]:
# Define the callback function for the scatter chart
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
     Input(component_id="payload-slider", component_property="value")]
)
def update_scatter_chart(selected_site, payload_range):
    if selected_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Correlation between Payload and Mission Outcome for All Sites',
                         labels={'class': 'Mission Outcome', 'Payload Mass (kg)': 'Payload Mass (kg)',
                                 'Booster Version Category': 'Booster Version'})
        return fig
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == selected_site) & 
                                (spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title=f'Correlation between Payload and Mission Outcome for {selected_site}',
                         labels={'class': 'Mission Outcome', 'Payload Mass (kg)': 'Payload Mass (kg)',
                                 'Booster Version Category': 'Booster Version'})
        return fig

In [6]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

# ** CREATED BY : MOHAMED MAHMOUD **